# Programmation linéaire en nombres entiers

In [ ]:
using LinearAlgebra
using JuMP, Gurobi

## Branch & Bound

Source: http://web.tecnico.ulisboa.pt/mcasquilho/compute/_linpro/TaylorB_module_c.pdf

Le propriétaire d'un atelier d'usinage prévoit de se développer en achetant de nouvelles machines - des presses et des tours. Le propriétaire a estimé que chaque presse achetée augmenterait les bénéfices de \\$100 par jour et que chaque tour augmenterait les bénéfices de \\$150 par jour. Le nombre de machines que le propriétaire peut acheter est limité par le coût des machines et l'espace au sol disponible dans le magasin. Les prix d'achat des machines et les espaces requis sont présentés ci-dessous.

| Machine | Espace requis au sol ($pi^2$) | Prix d'achat |
| --- | --- | --- |
| Presse | 15 | \$8.000 |
| Tour | 30 | \$4.000 |

Le propriétaire dispose d'un budget de \$40.000 pour l'achat de machines et de 200 pieds carrés d'espace au sol disponible. Le propriétaire veut savoir combien de chaque type de machine acheter pour maximiser l'augmentation quotidienne des bénéfices.

Travaillons avec la relaxtion linéaire

In [ ]:
m = Model(with_optimizer(Gurobi.Optimizer))

# Variables
@variable(m, x1 >= 0) # nombre de presses
@variable(m, x2 >= 0) # nombre de tours

# Fonction objectif
@objective(m, Max, 100x1 + 150x2)  # profit supplémentaire

# Contraintes
@constraint(m, constraint1, 8x1 +  4x2  <= 40)     # budget (en milliers de dollars)
@constraint(m, constraint2,  15x1 + 30x2  <= 200)  # contrainte d'espace

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

In [ ]:
obj = []

In [ ]:
function sol_output()
    if (termination_status(m) == MOI.OPTIMAL)
        println("Solution optimale:")
        println("x1 = ", value(x1), " x2 = ", value(x2))
        println("Valeur optimale = ", objective_value(m))
        
        # On va enregistrer les valeurs objectifs trouvées.
        append!(obj, objective_value(m))
    else
        println("Problème non réalisable")

        append!(obj, Inf)
    end
end

In [ ]:
sol_output()

La solution n'est pas entière. Nous devons donc brancher à partir de celle-ci.

Traçons l'arbre d'exploration en commançant par la racine que nous pouvons à présent créer.

In [ ]:
using Plots
using GraphRecipes
using AbstractTrees
using Printf

La fonction suivante permet d'illustrer le branchement à partir d'un noeud. Nous étiquettons les noeuds avec $P$ suivi du numéro du problème, et `father` reprendra le numéro du problème père.

In [ ]:
mutable struct bandb
    T:: Matrix
    labels
    colors
end

function add_problems(T, father::Int, labels, colors)
    (m,n) = size(T)
    A = zeros(m,2)
    T = hcat(T,A)
    A = zeros(2,m+2)
    T = vcat(T,A)
    T[father,n+1]=1
    T[father,n+2]=1
    label = "P$(m+1)\n "
    push!(labels, label)
    label = "P$(m+2)\n "
    push!(labels, label)
    
    # Note: nous devrions utiliser edgelabel pour mieux représenter
    # le branchements le long des arêtes de l'arbre.
    
    return T, labels, [colors colorant"#389826" colorant"#389826"]
end


function add_problems(tree:: bandb, father::Int)
    (m,n) = size(bb.T)
    A = zeros(m,2)
    bb.T = hcat(bb.T,A)
    A = zeros(2,m+2)
    bb.T = vcat(bb.T,A)
    bb.T[father,n+1]=1
    bb.T[father,n+2]=1
    label = "P$(m+1)\n "
    push!(tree.labels, label)
    label = "P$(m+2)\n "
    push!(tree.labels, label)

    # Note: nous devrions utiliser edgelabel pour mieux représenter
    # le branchements le long des arêtes de l'arbre.
    
    tree.colors = [colors colorant"#389826" colorant"#389826"]
end

In [ ]:
default(size=(600, 600))

colors = [colorant"#389826"]
T = zeros(1,1)
labels = ["P1\n$(@sprintf("%.2f", obj[1]))"];

In [ ]:
bb = bandb(zeros(1,1),["P1\n$(@sprintf("%.2f", obj[1]))"],[colorant"#389826"])

On va brancher sur la plus grande valeur fractionnaire, ici $x_2$.

In [ ]:
add_problems(bb, 1)
graphplot(bb.T, method=:tree, root=:top, names = bb.labels, nodeshape=:circle, nodesize=0.1,
          curves=false, linewidth=2,  axis_buffer=0.4, font=18)

In [ ]:
labels = bb.labels
colors = bb.colors
T = bb.T

In [ ]:
#T, labels, colors = add_problems(T, 1, labels, colors)
#graphplot(T, method=:tree, root=:top, names = labels, nodeshape=:circle, nodesize=0.1,
#          curves=false, linewidth=2,  axis_buffer=0.4, font=18)

Nous construisons P2 en ajoutant la contrainte $x_2 \leq 5$, tandis que P3 sera obtenu en prenant la contrainte $x_2 \geq 6$.

In [ ]:
@constraint(m, x2left, x2 <= 5)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

sol_output()
labels[2] = "P2\n$(@sprintf("%.2f", obj[2]))";

In [ ]:
delete(m, x2left)
@constraint(m, x2right, x2 >= 6)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

sol_output()
labels[3] = "P3\n$(@sprintf("%.2f", obj[3]))";

Pour $x_2 \leq 5$, la borne supérieure sur la fonction objectif est de 1000, mais la solution n'est pas entière.

Pour $x_2 \geq 6$, la borne supérieure sur la fonction objectif est de 1033+1/3, mais la solution n'est pas non plus entière.

Nous allons brancher à partir de la plus grande borne supérieure, correspondante au problème P3. Comme la solution optimale du problème relâxé est $x_1 = 4/3$ et $x_2 = 6.0$, nous devons brancher sur $x_1$.

In [ ]:
T, labels, colors = add_problems(T, 3, labels, colors)
graphplot(T, method=:tree, root=:top, names = labels, nodeshape=:circle, nodesize=0.1,
          curves=false, linewidth=2,  axis_buffer=0.4, font=18)

In [ ]:
@constraint(m, x1left, x1 <= 1)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

sol_output()
labels[4] = "P4\n$(@sprintf("%.2f", obj[4]))";

In [ ]:
delete(m, x1left)
@constraint(m, x1right, x1 >= 2)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

sol_output()
labels[5] = "P5\n$(@sprintf("%.2f", obj[5]))";

En ajoutant la contrainte $x_1 \geq 2$, le problème est non réalisable. Nous enregistrons la valeur Inf pour représenter l'infaisabilité.

Nous ne pouvons donc plus brancher de ce côté. Avec $x_1 \leq 1.0$, la solution n'est toujours pas entière. Nous allons brancher sur $x_2$ qui demeure fractionnaire.

In [ ]:
colors[5] = colorant"#CB3C33";

T, labels, colors = add_problems(T, 4, labels, colors)
graphplot(T, method=:tree, root=:top, names = labels, nodeshape=:circle, nodesize=0.1,
          curves=false, linewidth=2,  axis_buffer=0.4, font=18)

Branchons sur $x_2$.

In [ ]:
delete(m, x1right)
@constraint(m, x1left1, x1 <= 1)
@constraint(m, x2left6, x2 <= 6)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

sol_output()
labels[6] = "P6\n$(@sprintf("%.2f", obj[6]))";

La solution est à présent entière, aussi nous obtenons une borne inférieure sur la valeur optimale, et nous élaguons le noeud comme nous ne pouvons plus brancher dessus.

L'autre branche consiste à examiner le cas où $x_2 \geq 7$.

In [ ]:
delete(m, x2left6)
@constraint(m, x2right7, x2 >= 7)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

sol_output()
labels[7] = "P5\n$(@sprintf("%.2f", obj[7]))";

Le problème est non-réalisable. Nous pouvons élaguer le noeud.

In [ ]:
graphplot(T, method=:tree, root=:top, names = labels,
          nodeshape=:circle, nodesize=0.1, curves=false, linewidth=1,  axis_buffer=0.3, shorten=0.01, font=12)

Retournons au noeud non élagué avec $x_2 \leq 5$. Dans ce cas la borne supérieure sur la fonction objectif est de 1000, qui est la valeur trouvée précédemment avec la solution entière $x_1 = 1$, $x_2 = 6$. Nous ne pouvons donc pas améliorer l'objectif en explorant le cas $x_2 <= 5$ et nous élaguons le noeud correspondant. Comme il n'y a plus de noeud non élagué, nous nous arrêtons avec la solution optimale $x_1 = 1$, $x_2 = 6$.

## Méthode de coupes

### Constuction d'une coupe de Gomory

* Étape 1: Arrondir inférieurement les coefficients du terme de gauche de la contrainte, ansi que du terme de droite.
* Étape 2: Remplacer le signe d'égalité par une inégalité plus petite que. 
* Étape 3: Soustraire la contrainte originale à la contrainte obtenue à l'étape 2. 

Prenons par exemple la contrainte
$$
2.3 x - .6 y = 3.9
$$
Les étapes 1 et 2 donnent
$$
2 x - y \leq 3
$$
L'étape 3 mène à la coupe
$$
-.3x - .4 y \leq -0.9
$$

### Exemple 1

Considérons le problème
\begin{align*}
\min\ & -21x_1 - 11x_2 \\
\text{sujet à } & 7x_1 + 4x_2 + x_3 = 13 \\
& x_1 \geq 0, x_2 \geq 0, x_3 \geq 0 \\
& x_1, x_2, x_3 \text{ entiers.}
\end{align*}
Nous allons tout d'abord le modélier à l'aide de JuMP, puis utiliser la méthode de coupes.

Nous reformulons le problème dans la modélisation JuMP en tenant compte de ses caractéristiques.

In [ ]:
m = Model(with_optimizer(Gurobi.Optimizer))

# Variables
@variable(m, x1 >=0, Int)
@variable(m, x2 >=0, Int)

# Fonction objectif
@objective(m, Max, 21x1 + 11x2)

# Contraintes
@constraint(m, constraint1, 7x1 +  4x2 <= 13)

# Modèle complet
print(m)

# Résoudre le problème
status = optimize!(m)

println("Optimal solution:")
println("x1 = ", value(x1), " x2 = ", value(x2))

Résolvons la relaxation linéaire avec la méthode du simplexe primal.

Écrivons le tableau initial
$$
\begin{array}{c|ccc|c}
    & x_1 & x_2 & x_3 & b \\
    \hline
    x_3 & 7 & 4 & 1 & 13 \\
    \hline
    & -21 & -11 & 0 & 0
\end{array}
$$
Le système est déjà sous forme canonique ($x_3$ peut être vu comme variable d'écart). En choisissant le coût le plus négatif, nous échangeons $x_1$ et $x_3$ pour obtenir le tableau
$$
\begin{array}{c|ccc|c}
    & x_1 & x_2 & x_3 & b \\
    \hline
    x_1 & 1 & 4/7 & 1/7 & 13/7 \\
    \hline
    & 0 & 1 & 3 & -39
\end{array}
$$
Tous les coûts réduits sont non-négatifs et donc la solution est optimale, mais ne satisfait pas les conditions d'intégralité.


Nous ajoutons la contrainte
$$
\left( \left\lfloor \frac{4}{7} \right\rfloor - \frac{4}{7} \right) x_2 + \left( \left\lfloor \frac{1}{7} \right\rfloor - \frac{1}{7} \right) x_3 \leq \left\lfloor \frac{13}{7} \right\rfloor - \frac{13}{7},
$$
ou de manière équivalente
$$
 - \frac{4}{7} x_2 - \frac{1}{7} x_3 + x_4 = -\frac{6}{7},
$$
avec $x_4 \geq 0$, $x_4$ entier.
Le problème devient
\begin{align*}
\min\ & -21x_1 - 11x_2 \\
\text{sujet à } & 7x_1 + 4x_2 + x_3 = 13 \\
& - \frac{4}{7} x_2 - \frac{1}{7} x_3 + x_4 = -\frac{6}{7} \\
& x_1 \geq 0, x_2 \geq 0, x_3 \geq 0, x_4 \geq 0 \\
& x_1, x_2, x_3, x_4 \text{ entiers.}
\end{align*}
Nous pardons du tableau précédent pour obtenir le tableau du simplexe du problème relaxé linéairement
$$
\begin{array}{c|cccc|c}
    & x_1 & x_2 & x_3 & x_4 & b \\
    \hline
    x_1 & 1 & 4/7 & 1/7 & 0 & 13/7 \\
    x_4 & 0 & -4/7 & -1/7 & 1 & -6/7 \\
    \hline
    & 0 & 1 & 3 & 0 & 39
\end{array}
$$
Les conditions d'optimalité sont satisfaites, mais pas de faisabilité, aussi nous exécutons une itération du simplexe dual pour rétablir la faisabilité.

Nous échangeons $x_2$ et $x_4$ pour obtenir
$$
\begin{array}{c|cccc|c}
    & x_1 & x_2 & x_3 & x_4 & b \\
    \hline
    x_1 & 1 & 0 & 0 & 1 & 1 \\
    x_2 & 0 & 1 & 1/4 & -7/4 & 3/2 \\
    \hline
    & 0 & 0 & 11/4 & 7/4 & 37.5
\end{array}
$$
La solution est à présent réalisable et optimale, mais $x_2$ est à présent fractionnaire, et nous allons construire une nouvelle coupe, en considérant l'inégalité
$$
\left( \left\lfloor \frac{1}{4} \right\rfloor - \frac{1}{4} \right) x_3 + \left( \left\lfloor \frac{-7}{4} \right\rfloor + \frac{7}{4} \right) x_4 \leq \left\lfloor \frac{3}{2} \right\rfloor - \frac{3}{2},
$$
ou de manière équivalente
$$
 - \frac{1}{4} x_3 - \frac{1}{4} x_4 + x_5 = -\frac{1}{2},
$$
avec $x_5 \geq 0$, $x_5$ entier.
Nous ajoutons cette contrainte et à nouveau effectuons une itération du simplexe dual pour rétablir la faisabilité. Nous partons du tableau
$$
\begin{array}{c|ccccc|c}
    & x_1 & x_2 & x_3 & x_4 & x_5 & b \\
    \hline
    x_1 & 1 & 0 & 0 & 1 & 0 & 1 \\
    x_2 & 0 & 1 & 1/4 & -7/4 & 0 & 3/2 \\
    x_5 & 0 & 0 & -1/4 & -1/4 & 1 & -1/2 \\
    \hline
    & 0 & 0 & 11/4 & 7/4 & 0 & 37.5
\end{array}
$$


Nous échangeona $x_4$ et $x_5$ pour aboutir au tableau
$$
\begin{array}{c|ccccc|c}
    & x_1 & x_2 & x_3 & x_4 & x_5 & b \\
    \hline
    x_1 & 1 & 0 & -1 & 0 & 4 & -1 \\
    x_2 & 0 & 1 & 2 & 0 & -7 & 5 \\
    x_5 & 0 & 0 & 1 & 1 & -4 & 2 \\
    \hline
    & 0 & 0 & 1 & 0 & 7 & 34
\end{array}
$$
Nous voyons que la solution est entière, mais toujours non réalisable, et nous avons une itération duale supplémentaire à effectuer.
$$
\begin{array}{c|ccccc|c}
    & x_1 & x_2 & x_3 & x_4 & x_5 & b \\
    \hline
    x_3 & -1 & 0 & 1 & 0 & -4 & 1 \\
    x_2 & 2 & 1 & 0 & 0 & 1 & 3 \\
    x_5 & 1 & 0 & 0 & 1 & 0 & 1 \\
    \hline
    & 1 & 0 & 0 & 0 & 11 & 33
\end{array}
$$
La solution est à présent réalisable, entière, et les conditions d'optimalité remplies. Nous avons finalement convergé!

La solution optimale est donc $x_1 = 0$, $x_2 = 3$, pour une valeur optimale de -33, ce qui est conforme à la solution fournie par Gurobi.

### Exemple 2

Adapté de https://people.maths.bris.ac.uk/~mayt/MATH32500/2007/code/IP/gomoryExample.pdf

Considérons le problème
\begin{align*}
\min\ & -3x_1 - 4x_2 \\
\text{t.q. } & 3x_1 - x_2 \leq 12 \\
& 3x_1 + 11x_2 \leq 66 \\
& x \in \mathbb{N}^2.
\end{align*}

Nous transformons les contraintes en contraintes d'égalité avec l'ajout des variables d'écart $s_1$ et $s_2$:
\begin{align*}
\min\ & -3x_1 - 4x_2 \\
\text{t.q. } & 3x_1 - x_2 + s_1 = 12 \\
& 3x_1 + 11x_2 + s_2 = 66 \\
& x \in \mathbb{N}^2 \\
& s \geq 0.
\end{align*}

La relaxation linéaire donne
\begin{align*}
\min\ & -3x_1 - 4x_2 \\
\text{t.q. } & 3x_1 - x_2 + s_1 = 12 \\
& 3x_1 + 11x_2 + s_2 = 66 \\
& x \geq 0 \\
& s \geq 0.
\end{align*}

Nous obtenons le tableau du simplexe
$$
\begin{array}{c|cccc|c}
    & x_1 & x_2 & s_1 & s_2 & b \\
    \hline
    s_1 & 3 & -1 & 1 & 0 & 12 \\
    s_2 & 3 & 11 & 0 & 1 & 66 \\
    \hline
    & -3 & -4 & 0 & 0 & 0
\end{array}
$$
En vue de simplifier les calculs, nous échangeons $x_1$ et $s_1$. Nous obtenons le tableau du simplexe
$$
\begin{array}{c|cccc|c}
    & x_1 & x_2 & s_1 & s_2 & b \\
    \hline
    x_1 & 1 & -1/3 & 1/3 & 0 & 4 \\
    s_2 & 0 & 12 & -1 & 1 & 54 \\
    \hline
    & 0 & -5 & 1 & 0 & 12
\end{array}
$$

Nous échangeons à présent $x_2$ et $s_2$.
$$
\begin{array}{c|cccc|c}
    & x_1 & x_2 & s_1 & s_2 & b \\
    \hline
    x_1 & 1 & 0 & 11/36 & 1/36 & 11/2 \\
    x_2 & 0 & 1 & -1/12 & 1/12 & 9/2 \\
    \hline
    & 0 & 0 & 7/12 & 5/12 & 69/2
\end{array}
$$

Tous les coûts réduits sont non-négatifs, mais la solution n'est pas entière.

Considérons la contrainte
$$
x_1 + \frac{11}{36}s_1 + \frac{1}{36}s_2 = \frac{11}{2}.
$$
L'inégalité
$$
x_1 \leq 5
$$
doit être satisfaire pour tout solution réalisable. En soustrayant la contrainte initiale, nous obtenons la coupe
$$
-\frac{11}{36}s_1 - \frac{1}{36}s_2 \leq -\frac{1}{2}.
$$
Sous forme d'égalité, en introduisant la variable d'écart $s_3 \geq 0$, la coupe devient
$$
-\frac{11}{36}s_1 - \frac{1}{36}s_2 + s_3 = -\frac{1}{2}.
$$
Le tableau du simplexe devient
$$
\begin{array}{c|ccccc|c}
    & x_1 & x_2 & s_1 & s_2 & s_3 & b \\
    \hline
    x_1 & 1 & 0 & 11/36 & 1/36 & 0 & 11/2 \\
    x_2 & 0 & 1 & -1/12 & 1/12 & 0 & 9/2 \\
    s_3 & 0 & 0 & -11/36 & -1/36 & 1 & -1/2 \\
    \hline
        & 0 & 0 & 7/12 & 5/12 & 0 & 69/2
\end{array}
$$
La solution n'est plus réalisable mais elle est dual-réalisable.

Nous devons entreprendre une itération du simplexe dual. Nous prenons comme élément de pivot la troisième ligne et troisième colonne comme $7/12*36/11 = 21/11 < 5/12*36 = 15$.
Multiplions la troisième ligne par $-36/11$. Cela donne
$$
\begin{array}{c|ccccc|c}
    & x_1 & x_2 & s_1 & s_2 & s_3 & b \\
    \hline
    x_1 & 1 & 0 & 11/36 & 1/36 & 0 & 11/2 \\
    x_2 & 0 & 1 & -1/12 & 1/12 & 0 & 9/2 \\
    s_3 & 0 & 0 & 1 & 1/11 & -36/11 & 18/11 \\
    \hline
        & 0 & 0 & 7/12 & 5/12 & 0 & 69/2
\end{array}
$$
Les calculs deviennent complexes! Reprenons la procédure de pivot

In [ ]:
function pivot!(M::Matrix, i::Int, j::Int)
    @assert M[i, j] != 0 "pivot sur coefficient nul"
    M[i, :] /= M[i, j]
    m, n = size(M)
    for k in 1:m
        if i != k
            coef = M[k, j]
            M[k, :] -= coef * M[i, :]
        end
    end
    return M
end

In [ ]:
A = [ 1 0 11//36  1//36   0 
      0 1 -1//12  1//12   0 
      0 0 -11//36 -1//36  1 ]
b = [ 11//2 ; 9//2 ; -1//2 ]
c = [ 0 0   7//12  5//12  0  69//2 ]
M = [ A b ; c ]

pivot!(M, 3, 3)

Tous les coûts réduits sont non-négatifs, mais la solution n'est toujours pas entière, même si $x_1 = 5$!

Construisons une coupe à partir de la seconde contrainte
$$
x_2 + \frac{1}{11}s_2 - \frac{3}{11}s_3 = \frac{51}{11}.
$$
Nous formons l'inégalité
$$
x_2 - s_3 \leq 4
$$
et lui soustrayons la contrainte initiale pour obtenir la coupe
$$
-\frac{1}{11}s_2 - \frac{8}{11}s_3 \leq -\frac{7}{11}.
$$
Avec $s_4 \geq 0$, cela donne
$$
-\frac{1}{11}s_2 - \frac{8}{11}s_3 + s_4 = -\frac{7}{11}.
$$
Ajoutons cette contrainte au système.

In [ ]:
(m,n) = size(M)
M = hcat(M, Matrix{Rational{Int64}}(zeros(m,1)))
M[1:m,n+1] = M[1:m,n]
M[1:m,n] = zeros(m,1)
M = vcat(M, Matrix{Rational{Int64}}(zeros(1,n+1)))
M[m+1,:] = M[m,:]
M[m,:] = [0 0 0 -1//11 -8//11 1 -7//11]
M

Nous effectuons une itération du simplexe duale.

In [ ]:
pivot!(M, 4, 5)

À nouveau la solution n'est pas entière. Construisons une coupe à partir de la troisième
$$
s_1+\frac{1}{2}s_2-\frac{9}{2}s_4 = \frac{9}{2}.
$$
Nous en tirons l'inégalité
$$
s_1-5s_4 \leq 4,
$$
et la coupe
$$
-\frac{1}{2}s_2-\frac{1}{2}s_4 + s_5 = -\frac{1}{2}.
$$
Le nouveau système est

In [ ]:
(m,n) = size(M)
M = hcat(M, Matrix{Rational{Int64}}(zeros(m,1)))
M[1:m,n+1] = M[1:m,n]
M[1:m,n] = zeros(m,1)
M = vcat(M, Matrix{Rational{Int64}}(zeros(1,n+1)))
M[m+1,:] = M[m,:]
M[m,:] = [ 0 0 0 -1//2 0 -1//2 1 -1//2]
M

Itération du simplexe dual:

In [ ]:
pivot!(M, 5, 4)

La solution n'est toujours pas entière!

Partons de la première contrainte
$$
x_1 + \frac{3}{2}s_4 - \frac{1}{4}s_5 = \frac{17}{4}.
$$
Nous obtenons l'inégalité
$$
x_1 + s_4 - s_5 \leq 4,
$$
et la coupe
$$
-\frac{1}{2}s_4 - \frac{3}{4}s_5 + s_6 = -\frac{1}{4},
$$
donnant le système augmenté

In [ ]:
(m,n) = size(M)
M = hcat(M, Matrix{Rational{Int64}}(zeros(m,1)))
M[1:m,n+1] = M[1:m,n]
M[1:m,n] = zeros(m,1)
M = vcat(M, Matrix{Rational{Int64}}(zeros(1,n+1)))
M[m+1,:] = M[m,:]
M[m,:] = [ 0 0 0 0 0 -1//2 -3//4 1 -1//4]
M

Itération du simplexe dual:

In [ ]:
pivot!(M, 6, 7)

La solution n'est toujours pas entière! Repartons de la première contrainte
$$
x_1 + \frac{5}{3}s_4 - \frac{1}{3}s_6 = \frac{13}{3}.
$$
Nous obtenons l'inégalité
$$
x_1 + s_4 - s_6 \leq 4,
$$
et la coupe
$$
-\frac{2}{3}s_4 - \frac{2}{3}s_6 + s_7 = -\frac{1}{3}.
$$
Le nouveau tableau du simplexe est

In [ ]:
(m,n) = size(M)
M = hcat(M, Matrix{Rational{Int64}}(zeros(m,1)))
M[1:m,n+1] = M[1:m,n]
M[1:m,n] = zeros(m,1)
M = vcat(M, Matrix{Rational{Int64}}(zeros(1,n+1)))
M[m+1,:] = M[m,:]
M[m,:] = [ 0 0 0 0 0 -2//3 0 -2//3 1 -1//3]
M

Pivot dual:

In [ ]:
pivot!(M, 7, 8)

Toujours pas! Continuons avec la première contrainte
$$
x_1 + 2s_2 - \frac{1}{2}s_7 = \frac{9}{2},
$$
donnant l'inégalité
$$
x_1 + 2s_2 - s_7 \leq 4,
$$
et la coupe
$$
- \frac{1}{2}s_7 + s_8 = - \frac{1}{2}.
$$
Le nouveau tableau est

In [ ]:
(m,n) = size(M)
M = hcat(M, Matrix{Rational{Int64}}(zeros(m,1)))
M[1:m,n+1] = M[1:m,n]
M[1:m,n] = zeros(m,1)
M = vcat(M, Matrix{Rational{Int64}}(zeros(1,n+1)))
M[m+1,:] = M[m,:]
M[m,:] = [ 0 0 0 0 0 0 0 0 -1//2 1 -1//2]
M

Pivot dual:

In [ ]:
pivot!(M, 8, 9)

La solution est optimale et entière! Nous avons enfin terminé, avec comme solution optimale $x_1^* = 5$, $x_2^* = 4$. La valeur optimale est de -31.

## Problèmes de transport

En utilisant la méthode du coin Nord-Ouest, déterminez une solution réalisable au problème de transport défini par
$$
\mathbf{a} = (20, 10, 35, 15),\ \boldsymbol{b} = (10, 25, 5, 15, 25).
$$

Nous pouvons facilement construire le tableau suivant
$$
\begin{array}{|c|c|c|c|c|c}
\hline
10 & 10 & & & & 20 \\
\hline
& 10 & & & & 10 \\ 
\hline
& 5 & 5 & 15 & 10 & 35 \\
\hline
& & & & 15 & 15 \\
\hline
10 & 25 & 5 & 15 & 25
\end{array}
$$